# Manufactured solutions for Stokes problem

When solving numerically the stokes problem 

$$
\begin{aligned}
-\Delta \boldsymbol{u} + \nabla p & = f \\
\nabla \cdot \boldsymbol{u} &= 0,
\end{aligned}
$$

we can construct manufactured solutions that satisfy the divergence free
constraint, by taking a solution $\boldsymbol{u}$ that is the curl of a scalar
field (in two-dimensions) or the curl of a vector field in three dimensions.

For two dimensions: 

$$
\begin{split}
\boldsymbol{u}_x & = \partial_y g \\
\boldsymbol{u}_y & = -\partial_x g
\end{split}
$$

which satisfy by construction $\nabla \cdot \boldsymbol{u} = 0$ for $g\in
C^2(\Omega)$.

We can then build a forcing term $f$ that would force the system to have this
exact solution, by using the equality $f = -\Delta \boldsymbol{u} + \nabla p$.

If we want to use these in input files for the `stokes` executable of the
`fsi-suite`, we must replace the `**` with `^`, and we must write them in a
format that deal.II understands. This is done in the following cells, where we
first compute symbolically the differential operators, and then produce an
output that is readable by deal.II.

In [ ]:
from sympy import *
import textwrap

In [ ]:
def write_parameters(precursor, p):
    # ux, uy, and p are the exact solution of the velocity and pressure
    ux = diff(precursor, y)
    uy = -diff(precursor, x)

    # lap_u_x and lap_u_y are minus the laplacian of ux and uy
    lap_u_x = - diff(ux, x, 2) - diff(ux, y, 2)
    lap_u_y = - diff(uy, x, 2) - diff(uy, y, 2)

    # fx and fy are the resulting forcing terms
    fx = lap_u_x + diff(p, x)
    fy = lap_u_y + diff(p, y)

    def prm(ux, uy, p, fx, fy):
        def to_prm(x):
            text = str(x).replace('**', '^')
            start_len = len("  set Forcing term   =  ")
            return ' \\\n'.join(textwrap.wrap(text, 80-start_len, break_long_words=False, 
                                             subsequent_indent=' '*start_len))

        print(
            "subsection Functions\n  set Exact solution = ", to_prm(ux), ";",
            to_prm(uy), ";", to_prm(p), "\n",
            " set Forcing term   = ", to_prm(fx), ";",
            to_prm(fy), "; 0", "\nend\n"
            )
        
    prm(ux, uy, p, fx, fy)

In [ ]:
# Spatial variables
x,y = symbols('x y')

# Precursor of th exact solution. We take the curl of this function to have a 
# divergence free velocity field
precursor = sin(pi*x)**2*sin(pi*y)**2
pressure = cos(2*pi*x)*cos(2*pi*y)

write_parameters(precursor, pressure)